In [15]:
import pandas as pd
import json
import numpy as np
import time

with open('data.json') as f:
    data = json.load(f)

%matplotlib qt

# Reading to dataframe
# Creating additional columns
gen = ((k, v['assetA']['bid'], v['assetA']['ask'], v['assetB']['bid'], v['assetB']['ask']) for k, v in data.items())
data_list = [x for x in gen]
df = pd.DataFrame(data_list, columns=['timestamp', 'assetA_bid', 'assetA_ask', 'assetB_bid', 'assetB_ask'])
df['delta_t'] = df['timestamp'].astype('uint64').diff()
df['trade_freq'] = df.apply (lambda row: 1/row['delta_t'], axis=1)
df['trade_freq_avg'] = df['trade_freq'].rolling(window=15).mean()
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
df = df.set_index('datetime')

In [18]:

w = 7
asset = 'B'
ax = df['asset'+asset+'_ask'].plot(style='.-', color='tab:orange')
df['asset'+asset+'_ask'].rolling(w).mean().shift(int(-w/2)).plot(ax = ax, style=':', color='tab:orange')
ax = df['asset'+asset+'_bid'].plot(style='.-', color='tab:blue')
df['asset'+asset+'_bid'].rolling(w).mean().shift(int(-w/2)).plot(ax = ax, style=':', color='tab:blue')



def compute_trades(df):
    # Very simple market making starategy
    # if not rising
    # spread = abs(trailing ask - training bid)
    # if bid >= trailing ask - 0.2*spread then SELL 

    # if not falling
    # spread = abs(trailing ask - training bid)
    # if ask <= trailing bid + 0.2*spread then BUY 

    w=5
    # Spread
    df['trailing_ask'] = df['ask'].rolling(w).mean()
    df['trailing_bid'] = df['bid'].rolling(w).mean()
    df['spread'] = (df['trailing_ask']-df['trailing_bid']).abs()
    df['gradient'] = (df['trailing_bid']+df['trailing_ask'])/(2*df['trailing_ask'])
    df['gradient'] = df['gradient'].diff()
    
    
    #print(df.head(10))

    def operation(row):
        operation = None
        rising_threshold = 0.05
        falling_threshold = 0.05
        if not row['gradient'] > rising_threshold:
            if row['bid'] > row['trailing_ask'] - 0.2*row['spread']:
                operation = 'sell'
        if not row['gradient'] < falling_threshold:
            if row['ask'] < row['trailing_ask'] + 0.2*row['spread']:
                operation = 'buy'

        return operation
    
    df['operation'] = df.apply(operation, axis = 1)
    print(df.head(10))



# analyze assetA
dfA = pd.DataFrame()
dfA['ask'] = df['assetA_ask']
dfA['bid'] = df['assetA_bid']
compute_trades(dfA)

# analyze assetB
dfB = pd.DataFrame()
dfB['ask'] = df['assetB_ask']
dfB['bid'] = df['assetB_bid']
compute_trades(dfB)

# constructing action df
dfOperations = pd.DataFrame()
dfOperations['ts'] = df['timestamp']
dfOperations['actionA'] = dfA['operation']
dfOperations['actionB'] = dfB['operation']

ask      bid  trailing_ask  trailing_bid  spread  \
datetime                                                                        
2020-01-01 00:00:03.078  7189.00  7188.89           NaN           NaN     NaN   
2020-01-01 00:00:06.371  7190.09  7190.05           NaN           NaN     NaN   
2020-01-01 00:00:06.459  7190.09  7190.05           NaN           NaN     NaN   
2020-01-01 00:00:06.474  7190.43  7190.05           NaN           NaN     NaN   
2020-01-01 00:00:06.575  7190.39  7190.05      7190.000      7189.818   0.182   
2020-01-01 00:00:07.242  7190.33  7189.92      7190.266      7190.024   0.242   
2020-01-01 00:00:07.344  7190.25  7189.92      7190.298      7189.998   0.300   
2020-01-01 00:00:07.444  7190.21  7189.92      7190.322      7189.972   0.350   
2020-01-01 00:00:07.545  7190.12  7189.92      7190.260      7189.946   0.314   
2020-01-01 00:00:07.649  7190.01  7189.92      7190.184      7189.920   0.264   

                         gradient operation  
datetime  

In [40]:
def produce_output(df):
    output = []
    last_trade = 0
    df = df[df.actionB.isin(['sell', 'buy']) | df.actionA.isin(['sell', 'buy'])]
    for index, row in df.iterrows():
        actions = []
        if row['actionA'] == 'sell':
            actions.append('sellA')
        if row['actionA'] == 'buy':
            actions.append('buyA')
        if row['actionB'] == 'sell':
            actions.append('sellB')
        if row['actionB'] == 'buy':
            actions.append('buyB')

        ts = int(row['ts'])

        if len(actions)>0 and ts-last_trade>=30*1000:
            output.append({"time": int(row['ts']), "actions": actions})
            last_trade = ts

    return output

import json
output = produce_output(dfOperations)
with open('output.json', 'w') as fp:
    json.dump(output, fp)
